In [2]:
import pdfkit

# ✅ Step 1: Specify path to wkhtmltopdf.exe
path_to_wkhtmltopdf = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe'
config = pdfkit.configuration(wkhtmltopdf=path_to_wkhtmltopdf)

# ✅ Step 2: Create some HTML content
html_content = """
<html>
<head><title>Test Report</title></head>
<body>
    <h1>Student Report</h1>
    <p><strong>Name:</strong> Aryan</p>
    <p><strong>Course:</strong> AI & Python</p>
    <p><strong>Score:</strong> 95%</p>
    <p><strong>Feedback:</strong> Excellent progress. Keep improving your code structure.</p>
</body>
</html>
"""

# ✅ Step 3: Define output PDF file path
output_path = "test_student_report.pdf"

# ✅ Step 4: Generate PDF
pdfkit.from_string(html_content, output_path, configuration=config)

print("✅ PDF generated successfully:", output_path)


✅ PDF generated successfully: test_student_report.pdf


In [2]:
import os
import pandas as pd
import openai
import pdfkit
from jinja2 import Environment, FileSystemLoader
from datetime import datetime
import logging
import time

# --------------------------
# CONFIGURATION
# --------------------------

# Load API keys from environment variables
groq_api_key = os.getenv("GROQ_API_KEY")           # Set this in your environment
openai_api_key = os.getenv("OPENAI_API_KEY")       # Set this in your environment

# Use OpenAI by default (set to "groq" if using Groq)
LLM_PROVIDER = "openai"
MODEL_NAME = "gpt-3.5-turbo"  # or "mixtral-8x7b-32768" for Groq

# Set the correct key
if LLM_PROVIDER == "groq":
    import groq
    groq.api_key = groq_api_key
else:
    openai.api_key = openai_api_key

# Paths
CSV_PATH = "studentdata.csv"
TEMPLATE_FOLDER = "templates"
TEMPLATE_NAME = "report_template.html"
OUTPUT_FOLDER = "output_reports"

# Logging setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --------------------------
# SETUP
# --------------------------

# Load student data
df = pd.read_csv(CSV_PATH)

# Set up Jinja2 template
env = Environment(loader=FileSystemLoader(TEMPLATE_FOLDER))
template = env.get_template(TEMPLATE_NAME)

# Ensure output folder exists
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# --------------------------
# HELPER: Generate Feedback
# --------------------------
def generate_feedback(student_row):
    prompt = f"""
    You are an expert education counselor. Provide personalized feedback based on this learner's profile:

    Name: {student_row['Name']}
    Course: {student_row.get('Course', 'N/A')}
    Course Completion: {student_row.get('Completion_Percent', 'N/A')}%
    Assessment Score: {student_row.get('Assessment_Score', 'N/A')}
    Feedback Rating: {student_row.get('Feedback_Rating', 'N/A')}

    Write a short and constructive feedback (around 100 words) that includes praise and at least one area of improvement.
    """
    try:
        if LLM_PROVIDER == "groq":
            response = groq.ChatCompletion.create(
                model=MODEL_NAME,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=150
            )
        else:
            response = openai.ChatCompletion.create(
                model=MODEL_NAME,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=150
            )
        return response['choices'][0]['message']['content'].strip()

    except Exception as e:
        logging.error(f"Error generating feedback for {student_row['Name']}: {e}")
        return "Feedback unavailable due to API error."

# --------------------------
# MAIN LOOP
# --------------------------

for idx, row in df.iterrows():
    try:
        feedback = generate_feedback(row)
        date_today = datetime.today().strftime('%Y-%m-%d')

        html_content = template.render(student=row, feedback=feedback, date=date_today)
        output_path = os.path.join(OUTPUT_FOLDER, f"{row['Name']}_report.pdf")

        pdfkit.from_string(html_content, output_path)
        logging.info(f"Generated report: {output_path}")
        time.sleep(1)  # Respect API rate limits

    except Exception as e:
        logging.error(f"Failed to process student: {row.get('Name', 'Unknown')}, Error: {e}")

print("✅ All PDF reports generated successfully.")



2025-07-07 00:13:29,103 - ERROR - Error generating feedback for Jasmine_0: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://onboard.openai.com for details, or email support@openai.com if you have any questions.
2025-07-07 00:13:29,144 - ERROR - Failed to process student: Jasmine_0, Error: No wkhtmltopdf executable found: "b''"
If this file exists please check that this process can read it or you can pass path to it manually in method call, check README. Otherwise please install wkhtmltopdf - https://github.com/JazzCore/python-pdfkit/wiki/Installing-wkhtmltopdf
2025-07-07 00:13:29,144 - ERROR - Error generating feedback for Jasmine_1: No API key provided. You can set your API key in code using 'openai.api

KeyboardInterrupt: 

In [ ]:
import os
import pandas as pd
import pdfkit
from jinja2 import Environment, FileSystemLoader
import groq

# Set up GROQ client (replace with your GROQ key)
groq_client = groq.Client(api_key="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")
# Load student data
df = pd.read_csv('studentdata.csv')

# Set up Jinja2 environment for HTML templates
env = Environment(loader=FileSystemLoader('templates'))
template = env.get_template('report_template.html')

# Ensure output directory exists
output_dir = 'output_reports'
os.makedirs(output_dir, exist_ok=True)

def generate_feedback(student_row):
    prompt = f"""
    You are an expert education counselor. Provide personalized feedback based on this learner's profile:

    Name: {student_row['Name']}
    Course: {student_row.get('Course', 'N/A')}
    Course Completion: {student_row.get('Completion_Percent', 'N/A')}%
    Assessment Score: {student_row.get('Assessment_Score', 'N/A')}
    Feedback Rating: {student_row.get('Feedback_Rating', 'N/A')}

    Write a short and constructive feedback (around 100 words) that includes praise and at least one area of improvement.
    """
    response = groq_client.chat.completions.create(
        model="mixtral-8x7b-32768",  # Use a Groq-supported model
        messages=[{"role": "user", "content": prompt}],
        max_tokens=150
    )
    return response.choices[0].message.content.strip()

for idx, row in df.iterrows():
    feedback = generate_feedback(row)
    html_content = template.render(student=row, feedback=feedback)
    pdf_path = os.path.join(output_dir, f"{row['Name']}_report.pdf")
    pdfkit.from_string(html_content, pdf_path)

print("PDF reports generated for all learners.")

BadRequestError: Error code: 400 - {'error': {'message': 'The model `mixtral-8x7b-32768` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}

This error means the model name `"gpt-3.5-turbo"` is not available or not supported by the Groq API you are using.

**How to fix:**

1. **Check Groq's supported model names.**  
   Groq may use different model names than OpenAI. Common Groq-supported models are:
   - `"llama2-70b-4096"`
   - `"mixtral-8x7b-32768"`
   - `"gemma-7b-it"`

2. **Update your code to use a supported model.**  
   For example, to use Llama 2:
   ````python
   response = groq_client.chat.completions.create(
       model="llama2-70b-4096",
       messages=[{"role": "user", "content": prompt}],
       max_tokens=150
   )
   ````

**Summary:**  
Replace `"gpt-3.5-turbo"` with a model name supported by Groq. Check Groq's documentation or dashboard for the latest list of available models.

You are passing the **actual API key** as the environment variable name to `os.getenv`, which is incorrect.  
`os.getenv("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")` looks for an environment variable with that long name, which almost certainly does not exist.

**How to fix:**  
You should either:
- Store your API key in an environment variable named `GROQ_API_KEY` and use:
    ````python
    groq_client = groq.Client(api_key=os.getenv("GROQ_API_KEY"))
    ````
- Or, if you want to hardcode the key (not recommended for production), pass it directly:
    ````python
    groq_client = groq.Client(api_key="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")
    ````

**Summary:**  
Replace your current line with one of the above.  
**Do not** use your API key as the argument to `os.getenv`. Use the environment variable name instead.